In [5]:
%load_ext autoreload
%autoreload 2
import torch
import torchvision
import numpy as np
import os
from torch.utils.data import DataLoader
from torchvision.transforms import v2
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tqdm
from torch.distributions import Multinomial, Bernoulli
from tensorboard_logger import configure, log_value
from utils import utils
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
vit_b_16 = torchvision.models.vit_b_16(weights='ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1')
agent = utils.get_model()
checkpoint = torch.load("/media/samar/HDD1T/rl/PatchDrop-main/cv_dir/pretrain_ppo/ckpt_E_0_A_0.680_R_4.88E-01")
agent.load_state_dict(checkpoint['agent'])
agent.eval().cuda()
vit_b_16.eval().cuda();
mappings, _, patch_size = utils.action_space_model()
lr_size = 96

In [7]:
val_transform = v2.Compose([
    v2.PILToTensor(),
    v2.ToDtype(torch.float32,scale=True),
    v2.Resize(size=384,interpolation=torchvision.transforms.InterpolationMode.BICUBIC),
    v2.CenterCrop(384),
    v2.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [9]:
valdata = torchvision.datasets.ImageFolder(root="/media/samar/HDD1T/rl/PatchDrop-main/50-dataset/val",transform=val_transform)
valloader = DataLoader(valdata,batch_size=16,num_workers = 8,shuffle=False)

In [10]:
average_patch_use = 0
with torch.no_grad():
    for batch_idx, (inputs, targets) in tqdm.tqdm(enumerate(valloader), total=len(valloader)):

        inputs, targets = Variable(inputs), Variable(targets).cuda(non_blocking=True)
        
        inputs = inputs.cuda()

        # Get the low resolution agent images
        inputs_agent = inputs.clone()
        inputs_agent = torch.nn.functional.interpolate(inputs_agent, (lr_size, lr_size))
        probs = F.sigmoid(agent.forward(inputs_agent))

        # Sample the test-time policy
        policy = probs.data.clone()
        policy[policy<0.5] = 0.0
        policy[policy>=0.5] = 1.0
        # Get the masked high-res image and perform inference
        inputs = utils.agent_chosen_input(inputs, policy, mappings, patch_size)
        preds = vit_b_16(inputs)
        patch_use = policy.sum(1).float() / policy.size(1)
        _, pred_idx = preds.max(1)
        match = (pred_idx==targets).data
        patch_use_true_preds = patch_use[match]
        average_patch_use+=patch_use_true_preds.sum()

print(average_patch_use/len(valloader))

100%|██████████| 3125/3125 [38:12<00:00,  1.36it/s]

tensor(2.3340, device='cuda:0')


In [11]:
print((average_patch_use / (len(valloader)*128)) * 16)

tensor(0.2918, device='cuda:0')
